In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from kerastuner.engine.hypermodel import HyperModel
from kerastuner.tuners import RandomSearch


/var/folders/vz/m4r4czw95s9f09jw_6hwnf840000gn/T/ipykernel_15413/847426599.py:8: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.engine.hypermodel import HyperModel


In [2]:
img_height = 150
img_width = 150
batch_size = 32
data_directory = '/Users/baudi/AI/practicas/uvas/data/train_val/'
test_data_directory = '/Users/baudi/AI/practicas/uvas/data/test/'
validation_split = 0.2
seed = 42
num_classes = 4

# Función personalizada para ajustar el contraste
def contrast_adjustment(image, alpha=1.5, beta=0.0):
    return np.clip(alpha * image + beta, 0, 255)


# Crear un generador de imágenes con Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=(0.8, 1.2),  # Ajuste de brillo (0.8 - 1.2)
    preprocessing_function=contrast_adjustment,  # Ajuste de contraste
    validation_split=validation_split
)

#Crear un generador de imágenes para el conjunto de validación sin Data Augmentation
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=validation_split)

# Cargar conjuntos de entrenamiento y validación
train_dataset = train_datagen.flow_from_directory(
    data_directory,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse',
    subset='training',
    shuffle=True,
    seed=seed
)

val_dataset = val_datagen.flow_from_directory(
    data_directory,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation',
    shuffle=False,
    seed=seed
)

# Crear un generador de imágenes para el conjunto de pruebas sin Data Augmentation
test_datagen = ImageDataGenerator(rescale=1./255)
test_dataset = test_datagen.flow_from_directory(
    test_data_directory,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False,
    seed=seed
)


Found 2600 images belonging to 4 classes.
Found 648 images belonging to 4 classes.
Found 814 images belonging to 4 classes.


In [3]:

class MyModel(HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        model = models.Sequential()
        model.add(layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)))
        
        for i in range(hp.Int('num_conv_layers', min_value=1, max_value=3)):
            model.add(layers.Conv2D(
                filters=hp.Choice(f'filters_layer_{i}', values=[16, 32, 64]),
                kernel_size=3,
                padding='same',
                activation='relu'))
            model.add(layers.BatchNormalization())  # Capa de Batch Normalization
            model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        
        model.add(layers.Flatten())
        
        for i in range(hp.Int('num_dense_layers', min_value=1, max_value=2)):
            model.add(layers.Dense(
                units=hp.Choice(f'units_layer_{i}', values=[128, 256]),
                activation='relu'))
        
        model.add(layers.Dense(num_classes, activation='softmax'))

        learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                    metrics=['accuracy'])

        return model

my_model = MyModel(num_classes)


In [4]:
tuner = RandomSearch(
    my_model,
    objective='val_accuracy',
    max_trials=10,  # número de modelos para probar
    directory='/Users/baudi/AI/practicas/uvas/code/my_dir',
    project_name='uvas'
)

# Búsqueda de hiperparámetros
tuner.search(train_dataset, validation_data=val_dataset, epochs=20)

# Obtener el mejor modelo
best_model = tuner.get_best_models(num_models=1)[0]

# Entrenar el mejor modelo con todos los datos
history = best_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20
)

# Evaluación del modelo
val_loss, val_accuracy = best_model.evaluate(val_dataset)
print(f'Validation loss: {val_loss}, Validation accuracy: {val_accuracy}')


Trial 10 Complete [00h 02m 44s]
val_accuracy: 0.6774691343307495

Best val_accuracy So Far: 0.9027777910232544
Total elapsed time: 00h 30m 21s
INFO:tensorflow:Oracle triggered exit
Epoch 1/20


2023-04-07 16:07:56.600180: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


82/82 [==============================] - ETA: 0s - loss: 0.0540 - accuracy: 0.9823

2023-04-07 16:08:03.670867: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


82/82 [==============================] - 8s 90ms/step - loss: 0.0540 - accuracy: 0.9823 - val_loss: 0.6281 - val_accuracy: 0.8812
Epoch 2/20
82/82 [==============================] - 7s 87ms/step - loss: 0.0715 - accuracy: 0.9785 - val_loss: 0.3597 - val_accuracy: 0.9290
Epoch 3/20
82/82 [==============================] - 7s 89ms/step - loss: 0.0756 - accuracy: 0.9758 - val_loss: 1.3080 - val_accuracy: 0.8179
Epoch 4/20
82/82 [==============================] - 7s 87ms/step - loss: 0.0563 - accuracy: 0.9815 - val_loss: 4.1605 - val_accuracy: 0.6235
Epoch 5/20
82/82 [==============================] - 7s 87ms/step - loss: 0.0376 - accuracy: 0.9862 - val_loss: 0.9555 - val_accuracy: 0.8657
Epoch 6/20
82/82 [==============================] - 7s 88ms/step - loss: 0.0761 - accuracy: 0.9765 - val_loss: 0.1132 - val_accuracy: 0.9630
Epoch 7/20
82/82 [==============================] - 7s 87ms/step - loss: 0.0466 - accuracy: 0.9846 - val_loss: 2.0024 - val_accuracy: 0.8318
Epoch 8/20
82/82 [======